In [ ]:
import os
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import imagesize
from torchvision import datasets, models, transforms
import seaborn as sns
from PIL import Image
from PIL.ImageFile import ImageFile
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.profiler import profile, record_function, ProfilerActivity
import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet
import time
import torch.nn.functional as F

In [ ]:
#vars
ImageFile.LOAD_TRUNCATED_IMAGES = True

#batch_size = 128
n_epochs = 20
validation_split = .15
images_for_train = 30000
print_every = 100

optimizer_name = "Adagrad"
batch_size = 64
lr = 0.001974473
model_name = "efficientnet_b3"

name_training = "efficientnet_b3_training_wi"
os.mkdir(name_training)

In [ ]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:

ImageFile.LOAD_TRUNCATED_IMAGES = True
print(device)
print(torch.cuda.get_device_name(3))

In [ ]:
data_path = "data"
#data_path = "D:\Studium\OneDrive - Hochschule Karlsruhe\own-pictures"

image = []
labels = []
width = []
height = []

for file in os.listdir(data_path):
    if file == 'glas':
        for c in os.listdir(os.path.join(data_path, file)):
            image.append(c)
            labels.append('glas')
            w, h = imagesize.get(data_path + "/" + file + "/" + c)
            width.append(w), height.append(h)
    if file == 'organic':
        for c in os.listdir(os.path.join(data_path, file)):
            image.append(c)
            labels.append('organic')
            w, h = imagesize.get(data_path + "/" + file + "/" + c)
            width.append(w), height.append(h)
    if file == 'paper':
        for c in os.listdir(os.path.join(data_path, file)):
            image.append(c)
            labels.append('paper')
            w, h = imagesize.get(data_path + "/" + file + "/" + c)
            width.append(w), height.append(h)
    if file == 'restmuell':
        for c in os.listdir(os.path.join(data_path, file)):
            image.append(c)
            labels.append('restmuell')
            w, h = imagesize.get(data_path + "/" + file + "/" + c)
            width.append(w), height.append(h)
    if file == 'wertstoff':
        for c in os.listdir(os.path.join(data_path, file)):
            image.append(c)
            labels.append('wertstoff')
            w, h = imagesize.get(data_path + "/" + file + "/" + c)
            width.append(w), height.append(h)

data = {'Images': image, 'labels': labels, 'width': width, 'height': height}
data = pd.DataFrame(data)
lb = LabelEncoder()
data['encoded_labels'] = lb.fit_transform(data['labels'])

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)
data = data.head(images_for_train)
tr, val = train_test_split(data, stratify=data.labels, test_size=validation_split)
tr.reset_index(drop=True)
val.reset_index(drop=True)
print(val)

In [ ]:
waste_types_df = tr[['encoded_labels', 'labels']].drop_duplicates().sort_values(by='encoded_labels').reset_index(drop=True)
garbage_types = {}
for i in range(0, len(waste_types_df)):
    garbage_types[i] = waste_types_df.iloc[i].labels
print(garbage_types)

In [ ]:
data

In [ ]:
from torchvision import transforms

transform_tr = transforms.Compose(
    [transforms.Resize((300,300), interpolation= transforms.InterpolationMode.BICUBIC),
     transforms.RandomHorizontalFlip(),
     transforms.RandomVerticalFlip(),
     transforms.RandomRotation(10),
     transforms.CenterCrop(256),

     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
transform_val = transforms.Compose(
    [transforms.Resize((256,256), interpolation= transforms.InterpolationMode.BICUBIC),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

In [ ]:
from torch.utils.data import Dataset

class GarbageDataset(Dataset):
    def __init__(self, img_data, img_path, transform=None):
        self.img_path = img_path
        self.transform = transform
        self.img_data = img_data

    def __len__(self):
        return len(self.img_data)

    def __getitem__(self, index):
        img_name = os.path.join(self.img_path, self.img_data.iloc[index].labels, self.img_data.iloc[index].Images)
        image = Image.open(img_name).convert('RGB')
        #image = image.resize((300, 300))
        label = torch.tensor(self.img_data.iloc[index].encoded_labels)
        if self.transform is not None:
            image = self.transform(image)
        return image, label, img_name

In [ ]:
from torch.utils.data import DataLoader

train_dataset = GarbageDataset(tr, data_path, transform_tr)
test_dataset = GarbageDataset(val, data_path, transform_val)

In [ ]:
import numpy as np

def img_display(img):
    MEAN = torch.tensor([0.485, 0.456, 0.406])
    STD = torch.tensor([0.229, 0.224, 0.225])

    img = img * STD[:, None, None] + MEAN[:, None, None]
    #img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

In [ ]:
def accuracy(out, labels):
    _, pred = torch.max(out, dim=1)
    return torch.sum(pred == labels).item()

In [ ]:
class SSWQ_V1(nn.Module):
    def __init__(self, num_classes, dropout=0.5, hidden_layer1=2048, hidden_layer2=2048):
        super(SSWQ_V1, self).__init__()
        self.dropout = dropout
        self.hidden_layer1 = hidden_layer1
        self.hidden_layer2 = hidden_layer2
        self.num_classes = num_classes

        self.conv0 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, padding=1)

        self.conv1_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv1_3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.conv1_4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.conv2_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.conv2_4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(512*7*7, self.hidden_layer1)
        self.fc2 = nn.Linear(self.hidden_layer1, self.hidden_layer2)
        self.fc3 = nn.Linear(self.hidden_layer2, self.num_classes)

    def forward(self, x):
        x = F.relu(self.conv0(x))
        x = self.maxpool(x)
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = F.relu(self.conv1_3(x))
        #x = F.relu(self.conv1_4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        #x = F.relu(self.conv2_3(x))
        #x = F.relu(self.conv2_4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3_1(x))
        x = F.relu(self.conv3_2(x))
        #x = F.relu(self.conv3_3(x))
        #x = F.relu(self.conv3_4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv4_2(x))
        x = F.relu(self.conv4_3(x))
        #x = F.relu(self.conv4_4(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, self.dropout) #dropout was included to combat overfitting
        x = F.relu(self.fc2(x))
        x = F.dropout(x, self.dropout)
        x = self.fc3(x)
        return x

In [ ]:
def get_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, len(waste_types_df))
    elif model_name == "alexnet":
        model = models.alexnet(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier = nn.Linear(in_features, len(waste_types_df))
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        in_features = model.classifier[0].in_features
        model.classifier = nn.Linear(in_features, len(waste_types_df))
    elif model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, len(waste_types_df))
    elif model_name == "resnet34":
        model = models.resnet34(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, len(waste_types_df))
    elif model_name == "efficientnet_b3":
        model = EfficientNet.from_pretrained("efficientnet-b3",num_classes=len(waste_types_df))
    elif model_name == "efficientnet_b4":
        model = EfficientNet.from_pretrained("efficientnet-b4",num_classes=len(waste_types_df))
    return model

In [ ]:

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=7, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=7, pin_memory=True)
def train_model(net, criterion, optimizer, num_epochs,step_size=7, gamma=0.1):
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    time_per_epoch = []
    val_loss = []
    val_acc = []
    train_loss = []
    train_acc = []
    valid_loss_min = np.Inf
    total_step = len(train_dataloader)
    
    for epoch in range(1, num_epochs + 1):
        start = time.time()
        running_loss = 0.0
        correct = 0
        total = 0
        print(f'Epoch {epoch}\n')
        for batch_idx, (data_, target_, _) in enumerate(train_dataloader):
            target_ = target_.type(torch.LongTensor)
            data_, target_ = data_.to(device, non_blocking=True), target_.to(device, non_blocking=True)
            # zero the parameter gradients
            for param in net.parameters():
                param.grad = None
            #optimizer.zero_grad()
            outputs = net(data_)
            loss = criterion(outputs, target_)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            _, pred = torch.max(outputs, dim=1)
            correct += torch.sum(pred == target_).item()
            total += target_.size(0)
            #if batch_idx % print_every == 0:
            #print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, batch_idx, total_step, loss.item()))
        train_acc.append(100 * correct / total)
        train_loss.append(running_loss / total_step)
        print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}')
        batch_loss = 0
        total_t = 0
        correct_t = 0
        with torch.no_grad():
            net.eval()
            for data_t, target_t, _ in (test_dataloader):
                target_t = target_t.type(torch.LongTensor)
                data_t, target_t = data_t.to(device), target_t.to(device)  # on GPU
                outputs_t = net(data_t)
                loss_t = criterion(outputs_t, target_t)
                batch_loss += loss_t.item()
                _, pred_t = torch.max(outputs_t, dim=1)
                correct_t += torch.sum(pred_t == target_t).item()
                total_t += target_t.size(0)
            val_acc.append(100 * correct_t / total_t)
            val_loss.append(batch_loss / len(test_dataloader))
            network_learned = batch_loss < valid_loss_min
            print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}\n')
            # Saving the best weight
            if network_learned:
                valid_loss_min = batch_loss
                torch.save(net.state_dict(), name_training + '/model.pt')
                print('Detected network improvement, saving current model')

        net.train()
        exp_lr_scheduler.step()
        end = time.time()
        time_per_epoch.append(round((end-start),2))
        print("Time for this epoch: " + str(round((end-start),2)) + "s")
    return val_loss, val_acc, train_loss, train_acc, time_per_epoch

In [ ]:
# Get pretrained model
model = get_model(model_name)
#model = SSWQ_V1(len(waste_types_df), dropout=0.5, 
                #hidden_layer1=512, hidden_layer2=2048)
model = model.to(device)

# Define criterion
criterion = nn.CrossEntropyLoss()

# Configure optimizer
optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
# Train a model
val_loss,val_acc, train_loss, train_acc, time_per_epoch = train_model(model, criterion, optimizer, num_epochs=n_epochs)

In [ ]:
fig_loss = plt.figure(figsize=(20, 10))
plt.title("Train - Validation Loss")
plt.plot(train_loss, label='train')
plt.plot(val_loss, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.legend(loc='best')
plt.savefig(name_training + '/loss.png', format='png',bbox_inches='tight')
plt.savefig(name_training + '/loss.pdf', format='pdf',bbox_inches='tight')

In [ ]:
fig_accuracy = plt.figure(figsize=(20, 10))
plt.title("Train - Validation Accuracy")
plt.plot(train_acc, label='train')
plt.plot(val_acc, label='validation')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('accuracy', fontsize=12)
plt.legend(loc='best')
plt.savefig(name_training + '/accuracy.png', format='png',bbox_inches='tight')
plt.savefig(name_training + '/accuracy.pdf', format='pdf',bbox_inches='tight')

In [ ]:
#net = SSWQ_V1(len(waste_types_df), dropout=0.5, 
                #hidden_layer1=512, hidden_layer2=2048)
#net = models.resnet34(pretrained=False)
net = EfficientNet.from_pretrained("efficientnet-b3",num_classes=len(waste_types_df))
#in_features = net.fc.in_features
#net.fc = nn.Linear(in_features, len(waste_types_df))
net.to(device)
net.load_state_dict(torch.load(name_training + '/model.pt'))
net.eval()

In [ ]:
dataiter = iter(test_dataloader)
images, labels, path = dataiter.next()

# Viewing data examples used for training
fig, axis = plt.subplots(3, 5, figsize=(15, 11))
with torch.no_grad():
    for ax, image, label, path in zip(axis.flat, images, labels, path):
        ax.grid(None)
        ax.set_facecolor((1,1,1))
        ax.imshow(img_display(image))  # add image
        image, label = image.to(device), label.to(device)
        imagetensor = image.unsqueeze(0)
        imagetensor.to(device)
        output = net(imagetensor)
        softmax = nn.LogSoftmax(1)
        output = softmax(output)
        outputnumpy = torch.exp(output).to('cpu').numpy()
        output = output.argmax()
        k = output.item() == label.item()
        color = 'green'
        highest_probability = str(round(100 * outputnumpy[0][output.item()], 2))
        string = str(garbage_types[output.item()]) + "\np = " + highest_probability + "%"
        if not k:
            color = 'red'
            string = str(garbage_types[output.item()]) + " (" + str(garbage_types[label.item()]) + ")\np = " + highest_probability + "%"
        ax.set_title(string, fontsize=15, color=color)  # add label

In [ ]:
a = []
b = []
dict = {
    'paper':[0,0,0],
    'wertstoff':[0,0,0],
    'restmuell':[0,0,0],
    'glas':[0,0,0],
    'organic':[0,0,0],
}
with torch.no_grad():
    for data_t, target_t, _ in test_dataloader:
        target_t = target_t.type(torch.FloatTensor)
        data_t, target_t = data_t.to(device), target_t.to(device)  # on GPU
        for image, label in zip(data_t, target_t):
            imagetensor = image.unsqueeze_(0)
            imagetensor.to(device)
            output = net(image)
            output = output.argmax()
            k = output.item() == label.item()
            a.append(k)
            b.append(garbage_types[label.item()])
            # Absolute Count
            dict[garbage_types[label.item()]][0] += 1
            if k: # True
                dict[garbage_types[label.item()]][1] += 1
            else: # False
                dict[garbage_types[label.item()]][2] += 1

In [ ]:
df = {"a":a, "b":b}
df = pd.DataFrame(df)
fig, ax1 = plt.subplots(figsize=(10, 10))
sns.countplot(x=df['b'], hue=df['a'], ax=ax1)
plt.savefig(name_training + '/countplot.png', format='png',bbox_inches='tight')
plt.savefig(name_training + '/countplot.pdf', format='pdf',bbox_inches='tight')

In [ ]:
print(dict)

In [ ]:
fig_time = plt.figure(figsize=(20, 10))
plt.title("Time per epoch")
plt.plot(time_per_epoch, label='time')
plt.xlabel('num_epochs', fontsize=12)
plt.ylabel('time in s', fontsize=12)
plt.legend(loc='best')
plt.savefig(name_training + '/fig_time.png', format='png',bbox_inches='tight')
plt.savefig(name_training + '/fig_time.pdf', format='pdf',bbox_inches='tight')

In [ ]:
sum = 0
for time in time_per_epoch:
     sum += time

avg_time = sum / len(time_per_epoch)
print(avg_time)